# [CSCI 3397/PSYC 3317] Lab 5: Machine Learning Overview

**Posted:** Friday, February 28, 2024

**Due:** Wednesday, March 13, 2024

__Total Points__: 14 pts

__Submission__: please rename the .ipynb file as __\<your_username\>_lab5a.ipynb__ before you submit it to canvas. Example: weidf_lab5.ipynb.

# <b>1. Linear regression</b>

In [ ]:
## utility functions

# dataset split
def data_split(N, ratio=[6,2,2]):
    # generate a shuffle array
    shuffle_idx = np.arange(N)
    np.random.shuffle(shuffle_idx)
    # divide into train-val-test by the ratio
    data_split = (np.cumsum(ratio)/float(sum(ratio))*N).astype(int)
    out_idx = [None] * len(ratio)
    out_idx[0] = shuffle_idx[:data_split[0]]
    for i in range(1,len(ratio)):
        out_idx[i] = shuffle_idx[data_split[i-1] : data_split[i]]
    return out_idx

def MSE(y,y_hat):
    # Lec. 9, page 19
    return ((y-y_hat)**2).mean()

## 1.1 Dataset Generation

In [ ]:
# data generation
import time
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(123)

# gt polynomial: y = 0.73x + 0.58
theta_gt = [0.58, 0.73]  # theta_0,theta_1
num_pt = 50

X = np.random.uniform(4, 10, num_pt).reshape(-1,1)
Y = theta_gt[0] + theta_gt[1] * X + 0.5 *np.random.normal(0, 1, num_pt).reshape(-1,1)

# for visualization
XX = np.linspace(4,10,100).reshape(-1,1)

train_idx, val_idx, test_idx = data_split(len(Y))

X_train, Y_train = X[train_idx], Y[train_idx]
X_val, Y_val = X[val_idx], Y[val_idx]
X_test, Y_test = X[test_idx], Y[test_idx]

plt.plot(X_train,Y_train,'ro')

## 1.2 Use 1-dim formula

Lec 17, page 20

In [ ]:
theta_1 = (X_train*Y_train).mean()-X_train.mean()*Y_train.mean()
theta_1 = theta_1/ ((X_train**2).mean()-X_train.mean()**2)
theta_0 = Y_train.mean()-theta_1*X_train.mean()

plt.plot(X_train,Y_train,'ro')
plt.plot(XX,XX*theta_1+theta_0,'b-')
plt.legend(['data', 'regression result'])

print('gt theta', theta_gt)
print('estimated theta (1-dim formula)', [theta_0,theta_1])


# evaluation on the training data
Y_train_hat = X_train*theta_1+theta_0
print('MSE error', MSE(Y_train, Y_train_hat))

## 1.3 Use N-dim formula

Lec 17, page 21

In [ ]:
X_train_aug = np.hstack([np.ones([len(X_train),1]),X_train])

theta_nd = np.linalg.solve(np.matmul(X_train_aug.T, X_train_aug), np.matmul(X_train_aug.T, Y_train))
print('estimated theta (N-dim formula)', theta_nd)

# evaluation on the training data
Y_train_hat = X_train*theta_nd[1]+theta_nd[0]
print('MSE error', MSE(Y_train, Y_train_hat))

# <b>2. Polynomial regression</b>
(Linear regression + polynomial feature)

## 2.1 Data generation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(123)

# gt polynomial: y = x^2 -10x +25
num_pt = 10

X2 = np.random.uniform(4, 10, num_pt).reshape(-1,1)
theta_gt2 = [25,-10,1]
Y2 = theta_gt2[0] + theta_gt2[1]*X2 +theta_gt2[2]*X2**2 + 0.5 *np.random.normal(0, 1, num_pt).reshape(-1,1)

# for visualization
XX2 = np.linspace(4,10,100).reshape(-1,1)

train_idx2, val_idx2, test_idx2 = data_split(len(Y2))

X_train2, Y_train2 = X2[train_idx2], Y2[train_idx2]
X_val2, Y_val2 = X2[val_idx2], Y2[val_idx2]
X_test2, Y_test2 = X2[test_idx2], Y2[test_idx2]

plt.plot(X_train2,Y_train2,'ro')

## 2.1 Use N-dim formula

Lec 18, page 18

In [ ]:
X_train2_aug = np.hstack([np.ones([len(X_train2),1]),X_train2,X_train2**2])

theta_nd2 = np.linalg.solve(np.matmul(X_train2_aug.T, X_train2_aug), np.matmul(X_train2_aug.T, Y_train2))
print('gt theta', theta_gt2)
print('estimated theta (N-dim formula)',theta_nd2)

# evaluation on the training data
Y_train2_hat = X_train2 * X_train2 * theta_nd2[2] + X_train2 * theta_nd2[1]+theta_nd2[0]
print('MSE error', MSE(Y_train2, Y_train2_hat))

# <b>3. Unsupervised learning </b>



## 3. Data: [[MedMNIST]](https://medmnist.com/)

In [ ]:
import os
if not os.path.exists('pathmnist.npz'):
  # download may take 8 min
  ! wget https://zenodo.org/record/5208230/files/pathmnist.npz

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def tensorTo2D(tensor):
    # only keep the first dimension: image index
    # reshape the rest dimensions into one dimension
    return tensor.reshape(tensor.shape[0], -1)

data = np.load('pathmnist.npz')
X_train = data['train_images'][::10]
Y_train = data['train_labels'][::10]
X_test = data['test_images'][::10]
Y_test = data['test_labels'][::10]

# ML assumes 2D input: N x feature input
X_train_2d = tensorTo2D(X_train)
X_test_2d = tensorTo2D(X_test)

num_label = Y_train.max() + 1

print('Train data size', X_train.shape)
print('Test data size', X_test.shape)
ui, uc = np.unique(Y_train, return_counts=True)
print('#label', num_label)
print('Train label counts', uc)

In [ ]:
plt.figure(figsize=(12, 12))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(X_train[i], cmap='gray')
    plt.axis('off')
    plt.title('class %d' % Y_train[i])

## 3.1 Clustering
Kmeans clustering method:
[[Sklearn documentation]](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)
[[Explanation]](https://www.machinecurve.com/index.php/2020/04/16/how-to-perform-k-means-clustering-with-python-in-scikit/)

Lec 17, Page 6-10

In [ ]:
# model training
from sklearn.cluster import KMeans
num_cluster = 5
max_iter = 10000
kmeans = KMeans(n_clusters = num_cluster, max_iter=max_iter)
cluster_labels = kmeans.fit_predict(X_train_2d)

In [ ]:
# visualization
num_disp = 4
plt.figure(figsize=(12, 12))
for i in range(num_cluster):
    index = np.where(cluster_labels==i)[0]
    for j in range(num_disp):
        plt.subplot(num_cluster, num_disp, i*num_disp+j+1)
        plt.imshow(X_train[index[j]])
        plt.axis('off')
        if j == 0:
            plt.title('cluster %d' % i)

## 3.2 Dimension Reduction

Principle component analysis (PCA) method:
[[Sklearn documentation]](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)
[[Example]](https://scikit-learn.org/stable/auto_examples/decomposition/plot_pca_iris.html)
[[Explanation]](https://towardsdatascience.com/a-one-stop-shop-for-principal-component-analysis-5582fb7e0a9c)

Lec. 17, Page 11-14

In [ ]:
# model training
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(X_train_2d)

# inference
X_train_pca2 = pca.transform(X_train_2d)

In [ ]:
# visualization
cc='rgbcykm'
plt.figure(figsize=(12, 12))
for label in range(len(cc)):
    index = Y_train[:,0]==label
    plt.plot(X_train_pca2[index,0], X_train_pca2[index,1], cc[label]+'.')

plt.legend(['cluster %d'%x for x in range(len(cc))])

# [14 pts] Exercise

## (1) [3 pts] Polynomial regressor for any order of K

- [2 pt] Build a function to do polynomial regression with the input order K (e.g., $\sum_{i=0}^{k}\theta_ix^i$) and return the estimated theta
- [1 pt] Sanity check: for K=2, print the MSE error for the train data (X_train2, Y_train2) in section 2 and check if the MSE values agree.

In [ ]:
# hint: create the feature in the beginning and use for-loop to fill in each feature dimension
def train_PR(x,y,k):
    ### Your code starts here

    ### Your code ends here
    # retun estimated theta
    return theta

theta_sanity = train_PR(X_train2, Y_train2, 2)
Y_sanity2 = X_train2 * X_train2 * theta_sanity[2] + X_train2 * theta_sanity[1] + theta_sanity[0]
print('sanity check MSE:', MSE(Y_train2, Y_sanity2))
print('Yes, MSE values agree; MSE is 0.246892141160028 for section 2 and my function.')

## (2) [5 pts] Model selection
Let `Ks=np.arange(1,11)`
- (a) [1 pt] For each K value, train a polynomial regression model with order=K, evaluate its MSE on the training data.
- (b) [1 pt] Draw a line-plot of the training MSE (`plt.plot(x,y,'-')`) and answer "which K shall we choose if the goal is to minimize the training error"
- (c) [1 pt] Evaluate the trained models (different K values) above on the validation data and answer "which K shall we choose if the goal is to minimize the validation error"
- (d) [1 pt] Repeat (c) on the test data as the "final/real-world" evaluation.
- (e) [1 pt] Which model selection criteria is better: minimize training error or validation error? Briefly explain why.

Lec. 18, slide 30

In [ ]:
### Your code starts here

## (a) train poly-K model (save the model parameter in a list) and print MSE


## (b) plot MSE vs. K


## (c) evaluate the K models above on the validation data



## (d) evaluate the K models above on the test data


## (e) explain


### Your code ends here

## (3) [6 pts] Clustering after dimension reduction

- [2 pts] repeat section 3 (PCA) for the training data with output dimension=10
- [2 pts] repeat section 2 (K-means with K=5) for the 10-dim features above for the training data
- [2 pts] Visualize the clustering result as in section 2

In [ ]:
### Your code starts here

## (a) PCA with output dim = 10



## (b) k-means on PCA-transformed X_train with K=5


## (c) visualize clustering result


### Your code ends here